# Data Science Capstone Project


## Purpose
To analyze the neighbourhoods of Waterloo and understand prospective Buisness oppurtunities after COVID-19 pandemic slows down. I am comparing the conclusion that I draw from waterloo which is a small city compared to toronto and see how Business oppurtunities would differ. If the same approach for toronto would differ from the approach in waterloo.


## Methodology
Getting data for waterloo neighborhoods.
Getting them displayed on a map properly.

## WIP - improvements

Notable TODOs:
- todo 1: Deciding what kind of data I need and get relavant data.
- todo 2: If correlation and conclusions can be drawn.
- todo 3: Tasty visualizations.
- Choropleth maps of neighbourhoods comparing number of venues.

## Results
Describe and comment the most important results.

## Suggested next steps
State suggested next steps, based on results obtained in this notebook.

# Setup

## Library import
We import all the required Python libraries

In [4]:
# Imported all the neccesary libraries 
from bs4 import BeautifulSoup # For Web Scraping

import csv
import pandas as pd #Data Analysis
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', None)
import numpy as np

import geocoder #get location data
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests
import json #To handle json files
from pandas.io.json import json_normalize

from sklearn.cluster import KMeans # To run k-means algorithm

import matplotlib.cm as cm
import matplotlib.colors as colors # matplotlib stuff
import matplotlib.pyplot as plt
#%matplotlib inline # backend for rendering plots wit

import folium # To visualize on map
from folium.map import *
from folium import plugins
from folium.plugins import MeasureControl
from folium.plugins import FloatImage

print('Libraries Imported')

Libraries Imported



# Data import
We retrieve all the required data for the analysis.

In [7]:
url = 'https://services.arcgis.com/ZpeBVw5o1kjit7LT/arcgis/rest/services/NeighbourhoodAssociations/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json'
data = requests.get(url).json()  


In [8]:
import json
with open('data.geojson', 'w') as json_file:
    json.dump(data, json_file)

In [9]:
address = 'Waterloo, Ontario, Canada'

geolocator = Nominatim(user_agent="waterloo_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Waterloo City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Waterloo City are 43.466874, -80.524635.


In [10]:
waterloo_map = folium.Map(location=[latitude, longitude], zoom_start=12)

waterloo_map


In [11]:
element = []

for item in data['features']:
    i = round(len(item['geometry']['rings'][0])/2)+round(len(item['geometry']['rings'][0])/3)
    element.append([item['attributes']['NAME'],item['attributes']['TYPE'],item['geometry']['rings'][0][i][1],item['geometry']['rings'][0][i][0]])
    
element[:5]

[['Uptown West', 'Neighbourhood Association', 43.457113845, -80.532676196],
 ['Mary Allen', 'Neighbourhood Association', 43.4592855160001, -80.511530793],
 ['Conservation Meadows', 'Neighbourhood', 43.4984582550001, -80.573241185],
 ['Upper Beechwood II', 'Homes Association', 43.4588916540001, -80.571388477],
 ['Beechwood North', 'Homes Association', 43.465506954, -80.563758648]]

In [12]:
df = pd.DataFrame(element,columns = ['Neighbourhood', 'Neighbourhood Type','lat','long'])

In [13]:
df.head()

,Neighbourhood,Neighbourhood Type,lat,long
0,Uptown West,Neighbourhood Association,43.457114,-80.532676
1,Mary Allen,Neighbourhood Association,43.459286,-80.511531
2,Conservation Meadows,Neighbourhood,43.498458,-80.573241
3,Upper Beechwood II,Homes Association,43.458892,-80.571388
4,Beechwood North,Homes Association,43.465507,-80.563759


In [14]:
# Displaying toronto areas by postal Code
for lat,lng,neighbourhood in zip(df['lat'],df['long'],df['Neighbourhood Type']):
    label = f'{neighbourhood}'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius = 5,
        popup=label,
        color = 'green',
        fill = True,
        parse_html=False, 
    ).add_to(waterloo_map)
    
# waterloo_map.add_child(MeasureControl())
FloatImage(url, bottom=5, left=85).add_to(waterloo_map)
waterloo_map.add_child(MeasureControl())
waterloo_map.save('index.html')

waterloo_map

Getting Kitchener Data

In [15]:
url = 'https://services1.arcgis.com/qAo1OsXi67t7XgmS/arcgis/rest/services/Neighbourhood_Association/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json'

In [16]:
data2 = requests.get(url).json() 

In [17]:
data2['features'][0]

{'attributes': {'OBJECTID': 1,
  'NEIGHBOURHOOD_ASSOCIATION': 'VICTORIA HILLS NEIGHBOURHOOD ASSOCIATION',
  'CONTACTNAME': 'PAULINE FREEMAN',
  'CONTACTADDRESS': 'C/O VICTORIA HILLS COMMUNITY CENTRE, 10 CHOPIN DR, KITCHENER ON N2M 5G2 vhna2011@gmail.com',
  'CONTACTPHONE': '519-741-2717',
  'WEB_URL': 'http://www.vhna.ca',
  'NEIGHBOURHOODASSOCIATIONID': 7724,
  'CREATE_DATE': 1242742777000,
  'UPDATE_DATE': 1589154182000,
  'SOURCE_DATE': 1518652800000,
  'SOURCE': '2018 SPREADSHEET',
  'MAPLABEL1': 'Victoria Hills N. A. (27)',
  'STATUS': 'ACTIVE',
  'CATEGORY': 'PRIMARY',
  'SUBCATEGORY': 'COMMUNITY CENTRE PROGRAMMING',
  'ASSOCIATION_NUMBER': 27,
  'ROTATION': -27,
  'Shape__Area': 2715948.44116211,
  'Shape__Length': 9487.15589335725},
 'geometry': {'rings': [[[-80.5172525812608, 43.4522940714914],
    [-80.5169949894019, 43.4518882179652],
    [-80.5169947152387, 43.4518877860998],
    [-80.5167422962754, 43.4514770213828],
    [-80.5166782435746, 43.4513801189444],
    [-80.5164

In [18]:
element = []

for item in data2['features']:
    i = round(len(item['geometry']['rings'][0])/5)
    element.append([item['attributes']['MAPLABEL1'],item['attributes']['NEIGHBOURHOOD_ASSOCIATION'],item['geometry']['rings'][0][i][1],item['geometry']['rings'][0][i][0]])
    
element[:5]

[['Victoria Hills N. A. (27)',
  'VICTORIA HILLS NEIGHBOURHOOD ASSOCIATION',
  43.43976762884,
  -80.5074309374848],
 ['King East N. A. (19)',
  'KING EAST NEIGHBOURHOOD ASSOCIATION',
  43.4463394228412,
  -80.4727877559867],
 ['Auditorium N. A. (2)',
  'AUDITORIUM NEIGHBOURHOOD ASSOCIATION',
  43.4456016219487,
  -80.4709589502376],
 ['Westmount N. A. (29)',
  'WESTMOUNT NEIGHBOURHOOD ASSOCIATION',
  43.4469953134838,
  -80.5220078007653],
 ['Bridgeport Community Assoc. (4)',
  'BRIDGEPORT COMMUNITY ASSOCIATION',
  43.4914432986357,
  -80.4809192238187]]

In [19]:
df2 = pd.DataFrame(element,columns = ['Neighbourhood', 'Neighbourhood Type','lat','long'])

In [20]:
df2.head()

,Neighbourhood,Neighbourhood Type,lat,long
0,Victoria Hills N. A. (27),VICTORIA HILLS NEIGHBOURHOOD ASSOCIATION,43.439768,-80.507431
1,King East N. A. (19),KING EAST NEIGHBOURHOOD ASSOCIATION,43.446339,-80.472788
2,Auditorium N. A. (2),AUDITORIUM NEIGHBOURHOOD ASSOCIATION,43.445602,-80.470959
3,Westmount N. A. (29),WESTMOUNT NEIGHBOURHOOD ASSOCIATION,43.446995,-80.522008
4,Bridgeport Community Assoc. (4),BRIDGEPORT COMMUNITY ASSOCIATION,43.491443,-80.480919


In [21]:
df_big = pd.concat([df,df2],axis = 0,sort = False).reset_index()

In [22]:
df_big.head()

,index,Neighbourhood,Neighbourhood Type,lat,long
0,0,Uptown West,Neighbourhood Association,43.457114,-80.532676
1,1,Mary Allen,Neighbourhood Association,43.459286,-80.511531
2,2,Conservation Meadows,Neighbourhood,43.498458,-80.573241
3,3,Upper Beechwood II,Homes Association,43.458892,-80.571388
4,4,Beechwood North,Homes Association,43.465507,-80.563759


In [23]:
address = 'Waterloo, Ontario, Canada'

geolocator = Nominatim(user_agent="waterloo_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Waterloo City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Waterloo City are 43.466874, -80.524635.


In [24]:
KW_map = folium.Map(location=[latitude, longitude], zoom_start=12)

KW_map

In [25]:
for lat,lng,neighbourhood in zip(df_big['lat'],df_big['long'],df_big['Neighbourhood Type']):
    label = f'{neighbourhood}'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius = 5,
        popup=label,
        color = 'green',
        fill = True,
        parse_html=False, 
    ).add_to(KW_map)
    
# waterloo_map.add_child(MeasureControl())
FloatImage(url, bottom=5, left=85).add_to(KW_map)
KW_map.add_child(MeasureControl())

KW_map

In [26]:
#KW2_map = folium.Map(location=[latitude, longitude], zoom_start=12)

# KW2_map
# for item in data['features']:
#     coords = item['geometry']['rings'][0]
#     foo = pd.DataFrame(coords,columns=['long','lat'])
#     for lat,lng in zip(foo['lat'],foo['long']):
#     #     lat = foo['lat'][0]
#     #     lng = foo['long'][0]
#         folium.CircleMarker(
#             [lat,lng],
#             radius = 0.5,
#             color = 'green',
#             fill = True,
#             parse_html=False, 
#         ).add_to(KW2_map)

# KW2_map
# foo['lat'][0]

In [27]:
lat = df_big['lat'][0]
long = df_big['long'][0]
radius = 500

CLIENT_ID = 'JR0WCEE3K2VYBHOHIRHCAPOL3BXLGM2WHQNPCXXJ0GB20HPP' #Foursquare ID
CLIENT_SECRET = '251PHKQSNFRZEAMTRWB3YKHYRHT50WRVAAW35AMS3CHXDFE2' # Foursquare Secret
VERSION = '20200516' # Foursquare API version
LIMIT = 500

url = f'https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={lat},{long}&radius={radius}&limit={LIMIT}'
json = requests.get(url).json()
json['response']['groups'][0]['items'][0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '5251b49a11d2c68b0e3b8a65',
  'name': "Fo'cheezy",
  'location': {'address': 'Erb st',
   'crossStreet': 'Canadian tire parking lot',
   'lat': 43.458229035211716,
   'lng': -80.53239871108174,
   'labeledLatLngs': [{'label': 'display',
     'lat': 43.458229035211716,
     'lng': -80.53239871108174}],
   'distance': 126,
   'cc': 'CA',
   'city': 'Waterloo',
   'state': 'ON',
   'country': 'Canada',
   'formattedAddress': ['Erb st (Canadian tire parking lot)',
    'Waterloo ON',
    'Canada']},
  'categories': [{'id': '4bf58dd8d48988d1cb941735',
    'name': 'Food Truck',
    'pluralName': 'Food Trucks',
    'shortName': 'Food Truck',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/streetfood_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-0-5251b49

In [28]:
for item in json['response']['groups'][0]['items']:
    print(item['venue']['location']['lat'])

43.458229035211716
43.4583
43.45505085892062
43.460940084963525
43.4589968265331
43.453840895506445
43.45445
43.461485624264505


In [29]:

names = df_big['Neighbourhood']
latitudes = df_big['lat']
longtitudes = df_big['long']
radius = 500

CLIENT_ID = 'JR0WCEE3K2VYBHOHIRHCAPOL3BXLGM2WHQNPCXXJ0GB20HPP' #Foursquare ID
CLIENT_SECRET = '251PHKQSNFRZEAMTRWB3YKHYRHT50WRVAAW35AMS3CHXDFE2' # Foursquare Secret
VERSION = '20200516' # Foursquare API version
LIMIT = 500

list = []


for name,lat,long in zip(names,latitudes,longtitudes):
    url = f'https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={lat},{long}&radius={radius}&limit={LIMIT}'
    json = requests.get(url).json()
    venues = json['response']['groups'][0]['items']
    try:
        list.append([(
                    name,
                    lat,
                    long,
                    v['venue']['name'], 
                    v['venue']['location']['lat'],
                    v['reasons']['items'][0]['type'],
                    v['venue']['location']['lng']) for v in venues]
                   )
    except:
        continue


In [30]:
list

[[('Uptown West',
   43.457113845,
   -80.532676196,
   "Fo'cheezy",
   43.458229035211716,
   'general',
   -80.53239871108174),
  ('Uptown West',
   43.457113845,
   -80.532676196,
   'Westervelt Brewery',
   43.4583,
   'general',
   -80.5323),
  ('Uptown West',
   43.457113845,
   -80.532676196,
   'Empire Public School',
   43.45505085892062,
   'general',
   -80.53526925225468),
  ('Uptown West',
   43.457113845,
   -80.532676196,
   'Lot188',
   43.460940084963525,
   'general',
   -80.53369998084021),
  ('Uptown West',
   43.457113845,
   -80.532676196,
   'GRT Bus Stop #1966',
   43.4589968265331,
   'general',
   -80.53753739753678),
  ('Uptown West',
   43.457113845,
   -80.532676196,
   'Economical Insurance - Cafeteria',
   43.453840895506445,
   'general',
   -80.5293706974652),
  ('Uptown West',
   43.457113845,
   -80.532676196,
   'Fountain',
   43.45445,
   'general',
   -80.5282),
  ('Uptown West',
   43.457113845,
   -80.532676196,
   '7 Star Convenience',
   43.461

In [29]:
# dataframe = pd.DataFrame([item for venue_list in list for item in venue_list],
#                          columns = ['Postal Code','Borough','lat','long','venue','venue lat','venue long','venue type'])

In [31]:
venue_stuff = pd.DataFrame([item for venue_list in list for item in venue_list],columns = ['Neighbourhood','lat','long','venue','venue lat','venue type','venue long'])

In [32]:
venue_stuff.head()

,Neighbourhood,lat,long,venue,venue lat,venue type,venue long
0,Uptown West,43.457114,-80.532676,Fo'cheezy,43.458229,general,-80.532399
1,Uptown West,43.457114,-80.532676,Westervelt Brewery,43.458300,general,-80.532300
2,Uptown West,43.457114,-80.532676,Empire Public School,43.455051,general,-80.535269
3,Uptown West,43.457114,-80.532676,Lot188,43.460940,general,-80.533700
4,Uptown West,43.457114,-80.532676,GRT Bus Stop #1966,43.458997,general,-80.537537


In [33]:
venue_stuff['venue'].value_counts()

Tim Hortons                                             17
Subway                                                   7
Starbucks                                                6
Domino's Pizza                                           6
TD Canada Trust                                          5
Dollarama                                                5
Shoppers Drug Mart                                       5
LCBO                                                     5
Petro-Canada                                             5
Food Basics                                              4
RBC Royal Bank                                           4
First Choice Haircutters                                 4
Dairy Queen                                              4
The Beer Store                                           3
Harvey's                                                 3
Glasgow Heights Plaza                                    3
Three Kretans                                           